# 인공지능 작사가 만들기

exploration 4th basic을 하며 Cloud shell에 ~/aiffel/lyricist/data 생성한 덕에 ~/aiffel/lyricist/data/lyrics에 데이터가 있다.

# 1. 함수 preprocess_sentence()에서 문장 길이 조절하기


1) 데이터 읽어오기

 : glob 모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이하다.
 
 : glob를 활용해 모든 txt파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장해보자


In [1]:
import glob
import os
import os, re
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

#여러개의 txt 파일을 모두 읽어서 raw_corpus에 담아주기
raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print("데이터 크기: ", len(raw_corpus))
print("Examples :\n", raw_corpus[:3])

데이터 크기:  187088
Examples :
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


2) 데이터 정제하기

 : preprocess_sentence() 함수 활용하기
 
 : 지나치게 긴 문장은 다른 데이터들이 과도한 padding을 갖게 하므로 제거한다.
 
 : 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외해보자

In [2]:
print(raw_corpus[:15])

["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?", 'It goes like this', 'The fourth, the fifth', 'The minor fall, the major lift', 'The baffled king composing Hallelujah Hallelujah', 'Hallelujah', 'Hallelujah', 'Hallelujah Your faith was strong but you needed proof', 'You saw her bathing on the roof', 'Her beauty and the moonlight overthrew her', 'She tied you', 'To a kitchen chair', 'She broke your throne, and she cut your hair']


In [3]:
#공백인 문장 지우기
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue
    if len(sentence.split()) >= 16: continue

In [4]:
#입력된 문장을
# 1. 소문자로 바꾸고, 양쪽 공백을 지운다.
# 2. 특수문자 양쪽에 공백을 넣고
# 3. 여러개의 공백은 하나의 공백으로 바꾼다.
# 4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾼다.
# 5. 다시 양쪽 공백을 지운다.
# 6. 문장 시작에는 <start>, 끝에는 <end>를 추가한다.
#이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있을 것이다.
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() #1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) #2
    sentence = re.sub(r'[" "]+', " ", sentence) #3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) #4
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>' #6
    return sentence

#정제 함수를 활용하여 정제된 문장 모으기
corpus = []

for sentence in raw_corpus:
    #우리가 원하지 않는 문장은 건너뛴다.
    if len(sentence) == 0: continue
    if len(sentence.split()) >= 16: continue
    
    #정제하고 담기
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
#정제된 결과를 10개 확인해보기
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

3) 평가 데이터셋 분리

 : tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하자.
 
  : 단어장의 크기는 12,000 이상 으로 설정하자
  
  : 총 데이터의 20% 를 평가 데이터셋으로 사용하자

In [5]:
def tokenize(corpus):
    #7000단어를 기억할 수 있는 tokenizer를 만들것이다.
    #우리는 이미 문장을 정제했으니 filters가 필요 없다.
    #7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀 것이다.
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 14000,
        filters = ' ',
        oov_token = '<unk>')
    
    #corpus를 이용하여 tokenizer 내부의 단어장을 완성한다.
    tokenizer.fit_on_texts(corpus)
    #준비한 tokenizer를 이용해 corpus를 Tensor로 변환한다.
    tensor = tokenizer.texts_to_sequences(corpus)
    #입력 데이터의 시퀀스 길이를 일정하게 맞춰 준다.
    #만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰준다.
    #문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용한다.
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2714 ...    0    0    0]
 [   2   34    7 ...   44    3    0]
 ...
 [   2  259  193 ...   12    3    0]
 [   5   22    9 ...   10 1100    3]
 [   2    7   33 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fca802d5f40>


In [6]:
#생성된 텐서 데이터를 3번째 행, 10번째 열까지만 출력
print(tensor[:3, :10])

[[   2   50    5   91  307   62   57    9  957 5745]
 [   2   17 2714  879    4    8   11 6178    6  347]
 [   2   34    7   35   15  161  283   28  335    4]]


In [7]:
#단어 사전 구축의 원리

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    
    if idx >= 10:break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [8]:
#생성된 텐서를 소스와 타겟으로 분리하여 모델이 학습할 수 있게 하자
#tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성한다.
#마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높다.
src_input = tensor[:, :-1]
#tensor에서 <start>를 잘라내서 타겟 문장을 생성한다.
tgt_input = tensor[:, 1:]

from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state = 42)

enc_train

array([[   2,   34, 1371, ...,    0,    0,    0],
       [   2,   52,    5, ...,    0,    0,    0],
       [   2,  100,  171, ...,    0,    0,    0],
       ...,
       [   2,    8, 1727, ...,    0,    0,    0],
       [   2,    5,   61, ...,    0,    0,    0],
       [   2,   34,    5, ...,    0,    0,    0]], dtype=int32)

In [9]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (134872, 14)
Target Train: (134872, 14)


4) 인공지능 만들기

In [10]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

#tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 총 7001개이다.
VOCAB_SIZE = tokenizer.num_words + 1

#준비한 데이터 소스로부터 데이터셋을 만든다.
#데이터셋에 대해서는 아래 문서를 참고하자
#자세히 알아둘 수록 도움이 많이 되는 중요한 문서이다.
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder = True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256 #워드 벡터의 차원수, 단어가 추상적으로 표현되는 크기이다.
hidden_size = 1024 #주어진 동일한 데이터를 보고 각자 생각을 한 뒤 결정을 내리게 하는 '일꾼'의 수
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

In [12]:
#데이터셋에서 데이터 한 배치만 불러오는 방법
for src_sample, tgt_sample in dataset.take(1): break
    
#한 배치만 불러온 데이터를 모델에 넣어보자
model(src_sample)

<tf.Tensor: shape=(256, 14, 14001), dtype=float32, numpy=
array([[[ 3.59002956e-06,  2.46070325e-04,  1.36797025e-04, ...,
          1.76139918e-04,  1.33142850e-04, -1.31033457e-05],
        [ 4.57876449e-05,  2.52802274e-04,  4.50991356e-04, ...,
          1.96772045e-04,  2.02789688e-05, -1.08874716e-04],
        [-1.65117701e-04,  1.46885548e-04,  7.58269220e-04, ...,
          3.28453869e-04, -3.34363198e-04,  4.07478074e-05],
        ...,
        [-4.22220008e-04, -4.69377002e-04, -3.49098700e-04, ...,
          1.25636265e-03, -4.27569603e-06,  2.20649221e-04],
        [-3.42795043e-04, -5.89451985e-04, -2.02339710e-04, ...,
          1.19847490e-03, -2.12579194e-04, -5.05906319e-05],
        [-2.32568476e-04, -4.39667492e-04, -1.32197165e-04, ...,
          1.11689209e-03, -2.66383460e-04, -3.61387589e-04]],

       [[ 3.59002956e-06,  2.46070325e-04,  1.36797025e-04, ...,
          1.76139918e-04,  1.33142850e-04, -1.31033457e-05],
        [ 1.40509010e-05,  3.94782510e-05, -2

In [13]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3584256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  14351025  
Total params: 31,574,961
Trainable params: 31,574,961
Non-trainable params: 0
_________________________________________________________________


5) 훈련하기

In [14]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer = optimizer)
model.fit(dataset, epochs = 10)

Epoch 1/10
658/658 [==============================] - 123s 184ms/step - loss: 3.5709
Epoch 2/10
658/658 [==============================] - 120s 182ms/step - loss: 3.1010
Epoch 3/10
658/658 [==============================] - 120s 183ms/step - loss: 2.9155
Epoch 4/10
658/658 [==============================] - 120s 182ms/step - loss: 2.7740
Epoch 5/10
658/658 [==============================] - 120s 182ms/step - loss: 2.6509
Epoch 6/10
658/658 [==============================] - 120s 182ms/step - loss: 2.5399
Epoch 7/10
658/658 [==============================] - 120s 182ms/step - loss: 2.4365
Epoch 8/10
658/658 [==============================] - 120s 182ms/step - loss: 2.3414
Epoch 9/10
658/658 [==============================] - 120s 182ms/step - loss: 2.2511
Epoch 10/10
658/658 [==============================] - 120s 182ms/step - loss: 2.1666


In [15]:
#밑에서 만드는 generate_text 함수는 모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행하게 한다.
def generate_text(model, tokenizer, init_sentence = "<start>", max_len=20):
    #테스트를 위해서 입력받은 init_sentence도 텐서로 변환한다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    #단어 하나씩 예측해 문장을 만든다
    # 1. 입력받은 문장의 텐서를 입력한다
    # 2. 예측된 값 중 가장 높은 확률인 word index를 뽑아낸다.
    # 3. 2에서 예측된 word index를 문장 뒤에 붙인다.
    # 4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마친다.
    while True:
        #1
        predict = model(test_tensor)
        #2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis = -1), axis = -1)[:,-1]
        #3
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis = 0)], axis=-1)
        #4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break
                
    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환한다.
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
        
    return generated

6) 결과물

In [16]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m better , i m better <end> '

In [17]:
generate_text(model, tokenizer, init_sentence="<start> i do", max_len=20)

'<start> i do not like them <end> '

In [18]:
generate_text(model, tokenizer, init_sentence="<start> christmas is", max_len=20)

'<start> christmas is the only one that i ve got <end> '

In [19]:
generate_text(model, tokenizer, init_sentence="<start> baby", max_len=20)

'<start> baby , baby , baby , baby , baby , baby <end> '

In [20]:
generate_text(model, tokenizer, init_sentence="<start> merry", max_len=20)

'<start> merry christmas <end> '

In [21]:
generate_text(model, tokenizer, init_sentence="<start> now", max_len=20)

'<start> now i m a survivor <end> '

In [22]:
generate_text(model, tokenizer, init_sentence="<start> see", max_len=20)

'<start> see i m a bad bitch <end> '

# 2. 함수 tokenize()에서 토큰 개수 조절하기

In [23]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

#여러개의 txt 파일을 모두 읽어서 raw_corpus에 담아주기
raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print("데이터 크기: ", len(raw_corpus))
print("Examples :\n", raw_corpus[:3])

데이터 크기:  187088
Examples :
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


In [24]:
#공백인 문장 지우기
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue
        
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() #1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) #2
    sentence = re.sub(r'[" "]+', " ", sentence) #3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) #4
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>' #6
    return sentence

#정제 함수를 활용하여 정제된 문장 모으기
corpus = []

for sentence in raw_corpus:
    #우리가 원하지 않는 문장은 건너뛴다.
    if len(sentence) == 0: continue
    
    #정제하고 담기
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

In [25]:
def tokenize(corpus):
    #7000단어를 기억할 수 있는 tokenizer를 만들것이다.
    #우리는 이미 문장을 정제했으니 filters가 필요 없다.
    #7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀 것이다.
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 14000,
        filters = ' ',
        oov_token = '<unk>')
    
    #corpus를 이용하여 tokenizer 내부의 단어장을 완성한다.
    tokenizer.fit_on_texts(corpus)
    #준비한 tokenizer를 이용해 corpus를 Tensor로 변환한다.
    tensor = tokenizer.texts_to_sequences(corpus)
    
    tensor = [x for x in tensor if len(x) <= 15]
    #입력 데이터의 시퀀스 길이를 일정하게 맞춰 준다.
    #만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰준다.
    #문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용한다.
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2643 ...    0    0    0]
 [   2   35    7 ...   43    3    0]
 ...
 [   2    5  107 ...    0    0    0]
 [   2  261  200 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fc953fd4af0>


In [26]:
src_input = tensor[:, :-1]
#tensor에서 <start>를 잘라내서 타겟 문장을 생성한다.
tgt_input = tensor[:, 1:]

from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state = 42)
enc_train

array([[   2,    8,    7, ...,    3,    0,    0],
       [   2,    5,   91, ...,    0,    0,    0],
       [   2,  106,  353, ...,    0,    0,    0],
       ...,
       [   2,   12,   68, ...,    0,    0,    0],
       [   2,  312,   23, ...,    0,    0,    0],
       [   2, 2165,    5, ...,    1,    3,    0]], dtype=int32)

In [27]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124981, 14)
Target Train: (124981, 14)


In [28]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

#tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 총 7001개이다.
VOCAB_SIZE = tokenizer.num_words + 1

#준비한 데이터 소스로부터 데이터셋을 만든다.
#데이터셋에 대해서는 아래 문서를 참고하자
#자세히 알아둘 수록 도움이 많이 되는 중요한 문서이다.
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder = True)
dataset


<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [29]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256 #워드 벡터의 차원수, 단어가 추상적으로 표현되는 크기이다.
hidden_size = 1024 #주어진 동일한 데이터를 보고 각자 생각을 한 뒤 결정을 내리게 하는 '일꾼'의 수
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

In [30]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer = optimizer)
model.fit(dataset, epochs = 10)

Epoch 1/10
610/610 [==============================] - 116s 183ms/step - loss: 3.4376
Epoch 2/10
610/610 [==============================] - 112s 183ms/step - loss: 2.9795
Epoch 3/10
610/610 [==============================] - 111s 182ms/step - loss: 2.8136
Epoch 4/10
610/610 [==============================] - 111s 182ms/step - loss: 2.6872
Epoch 5/10
610/610 [==============================] - 111s 182ms/step - loss: 2.5772
Epoch 6/10
610/610 [==============================] - 111s 182ms/step - loss: 2.4788
Epoch 7/10
610/610 [==============================] - 111s 182ms/step - loss: 2.3869
Epoch 8/10
610/610 [==============================] - 112s 182ms/step - loss: 2.3011
Epoch 9/10
610/610 [==============================] - 111s 182ms/step - loss: 2.2201
Epoch 10/10
610/610 [==============================] - 111s 182ms/step - loss: 2.1440


In [31]:
#밑에서 만드는 generate_text 함수는 모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행하게 한다.
def generate_text(model, tokenizer, init_sentence = "<start>", max_len=20):
    #테스트를 위해서 입력받은 init_sentence도 텐서로 변환한다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    #단어 하나씩 예측해 문장을 만든다
    # 1. 입력받은 문장의 텐서를 입력한다
    # 2. 예측된 값 중 가장 높은 확률인 word index를 뽑아낸다.
    # 3. 2에서 예측된 word index를 문장 뒤에 붙인다.
    # 4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마친다.
    while True:
        #1
        predict = model(test_tensor)
        #2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis = -1), axis = -1)[:,-1]
        #3
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis = 0)], axis=-1)
        #4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break
                
    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환한다.
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
        
    return generated

In [32]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m a mess <end> '

In [33]:
generate_text(model, tokenizer, init_sentence="<start> i do", max_len=20)

'<start> i do not like them <end> '

In [34]:
generate_text(model, tokenizer, init_sentence="<start> christmas is", max_len=20)

'<start> christmas is the moon <end> '

In [35]:
generate_text(model, tokenizer, init_sentence="<start> baby", max_len=20)

'<start> baby , baby , baby <end> '

In [36]:
generate_text(model, tokenizer, init_sentence="<start> merry", max_len=20)

'<start> merry christmas <end> '

In [37]:
generate_text(model, tokenizer, init_sentence="<start> now", max_len=20)

'<start> now i m a voodoo chile <end> '

In [38]:
generate_text(model, tokenizer, init_sentence="<start> see", max_len=20)

'<start> see i had to go back to the <unk> <end> '

# 3. embedding_size 와 hidden_size 값 바꿔보기

1) embedding_size = 256, hidden_size = 512

In [39]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print("데이터 크기: ", len(raw_corpus))
print("Examples :\n", raw_corpus[:3])

for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue
        
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() #1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) #2
    sentence = re.sub(r'[" "]+', " ", sentence) #3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) #4
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>' #6
    return sentence

corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
       
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 14000,
        filters = ' ',
        oov_token = '<unk>')
    
    tokenizer.fit_on_texts(corpus)
    
    tensor = tokenizer.texts_to_sequences(corpus)
    
    tensor = [x for x in tensor if len(x) <= 15]
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

src_input = tensor[:, :-1]

tgt_input = tensor[:, 1:]

from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state = 42)

BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder = True)

class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 512
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer = optimizer)
model.fit(dataset, epochs = 10)

데이터 크기:  187088
Examples :
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]
[[   2   50    5 ...    0    0    0]
 [   2   17 2643 ...    0    0    0]
 [   2   35    7 ...   43    3    0]
 ...
 [   2    5  107 ...    0    0    0]
 [   2  261  200 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fc91084a460>
Epoch 1/10
610/610 [==============================] - 51s 80ms/step - loss: 3.7359
Epoch 2/10
610/610 [==============================] - 50s 81ms/step - loss: 3.2434
Epoch 3/10
610/610 [==============================] - 49s 80ms/step - loss: 3.0230
Epoch 4/10
610/610 [==============================] - 50s 81ms/step - loss: 2.8878
Epoch 5/10
610/610 [==============================] - 50s 81ms/step - loss: 2.7866
Epoch 6/10
610/610 [==============================] - 50s 81ms/step - loss: 2.7004
Epoch 7/10
610/610 [==========================

2) embedding_size = 256, hidden_size = 256

In [40]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print("데이터 크기: ", len(raw_corpus))
print("Examples :\n", raw_corpus[:3])

for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue
        
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() #1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) #2
    sentence = re.sub(r'[" "]+', " ", sentence) #3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) #4
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>' #6
    return sentence

corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
       
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 14000,
        filters = ' ',
        oov_token = '<unk>')
    
    tokenizer.fit_on_texts(corpus)
    
    tensor = tokenizer.texts_to_sequences(corpus)
    
    tensor = [x for x in tensor if len(x) <= 15]
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

src_input = tensor[:, :-1]

tgt_input = tensor[:, 1:]

from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state = 42)

BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder = True)

class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 256
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer = optimizer)
model.fit(dataset, epochs = 10)

데이터 크기:  187088
Examples :
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]
[[   2   50    5 ...    0    0    0]
 [   2   17 2643 ...    0    0    0]
 [   2   35    7 ...   43    3    0]
 ...
 [   2    5  107 ...    0    0    0]
 [   2  261  200 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fc910195af0>
Epoch 1/10
610/610 [==============================] - 28s 43ms/step - loss: 3.8797
Epoch 2/10
610/610 [==============================] - 27s 44ms/step - loss: 3.3908
Epoch 3/10
610/610 [==============================] - 26s 43ms/step - loss: 3.2212
Epoch 4/10
610/610 [==============================] - 27s 43ms/step - loss: 3.0722
Epoch 5/10
610/610 [==============================] - 27s 44ms/step - loss: 2.9884
Epoch 6/10
610/610 [==============================] - 27s 44ms/step - loss: 2.9224
Epoch 7/10
610/610 [==========================

3) embedding_size = 256, hidden_size = 128

In [41]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print("데이터 크기: ", len(raw_corpus))
print("Examples :\n", raw_corpus[:3])

for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue
        
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() #1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) #2
    sentence = re.sub(r'[" "]+', " ", sentence) #3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) #4
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>' #6
    return sentence

corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
       
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 14000,
        filters = ' ',
        oov_token = '<unk>')
    
    tokenizer.fit_on_texts(corpus)
    
    tensor = tokenizer.texts_to_sequences(corpus)
    
    tensor = [x for x in tensor if len(x) <= 15]
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

src_input = tensor[:, :-1]

tgt_input = tensor[:, 1:]

from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state = 42)

BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder = True)

class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 128
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer = optimizer)
model.fit(dataset, epochs = 10)

데이터 크기:  187088
Examples :
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]
[[   2   50    5 ...    0    0    0]
 [   2   17 2643 ...    0    0    0]
 [   2   35    7 ...   43    3    0]
 ...
 [   2    5  107 ...    0    0    0]
 [   2  261  200 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fc9101d1670>
Epoch 1/10
610/610 [==============================] - 19s 28ms/step - loss: 4.1071
Epoch 2/10
610/610 [==============================] - 18s 29ms/step - loss: 3.4731
Epoch 3/10
610/610 [==============================] - 18s 29ms/step - loss: 3.3645
Epoch 4/10
610/610 [==============================] - 17s 28ms/step - loss: 3.2579
Epoch 5/10
610/610 [==============================] - 17s 28ms/step - loss: 3.1547
Epoch 6/10
610/610 [==============================] - 18s 29ms/step - loss: 3.0578
Epoch 7/10
610/610 [==========================

4) embedding_size = 128, hidden_size = 1024

In [43]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print("데이터 크기: ", len(raw_corpus))
print("Examples :\n", raw_corpus[:3])

for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue
        
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() #1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) #2
    sentence = re.sub(r'[" "]+', " ", sentence) #3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) #4
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>' #6
    return sentence

corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
       
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 14000,
        filters = ' ',
        oov_token = '<unk>')
    
    tokenizer.fit_on_texts(corpus)
    
    tensor = tokenizer.texts_to_sequences(corpus)
    
    tensor = [x for x in tensor if len(x) <= 15]
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

src_input = tensor[:, :-1]

tgt_input = tensor[:, 1:]

from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state = 42)

BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder = True)

class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 128
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer = optimizer)
model.fit(dataset, epochs = 10)

데이터 크기:  187088
Examples :
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]
[[   2   50    5 ...    0    0    0]
 [   2   17 2643 ...    0    0    0]
 [   2   35    7 ...   43    3    0]
 ...
 [   2    5  107 ...    0    0    0]
 [   2  261  200 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fc91084a7c0>
Epoch 1/10
610/610 [==============================] - 115s 181ms/step - loss: 3.4776
Epoch 2/10
610/610 [==============================] - 111s 182ms/step - loss: 3.0322
Epoch 3/10
610/610 [==============================] - 111s 182ms/step - loss: 2.8635
Epoch 4/10
610/610 [==============================] - 111s 182ms/step - loss: 2.7386
Epoch 5/10
610/610 [==============================] - 111s 182ms/step - loss: 2.6344
Epoch 6/10
610/610 [==============================] - 111s 182ms/step - loss: 2.5428
Epoch 7/10
610/610 [==============

5) embedding_size = 64, hidden_size = 1024

In [44]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print("데이터 크기: ", len(raw_corpus))
print("Examples :\n", raw_corpus[:3])

for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue
        
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() #1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) #2
    sentence = re.sub(r'[" "]+', " ", sentence) #3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) #4
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>' #6
    return sentence

corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
       
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 14000,
        filters = ' ',
        oov_token = '<unk>')
    
    tokenizer.fit_on_texts(corpus)
    
    tensor = tokenizer.texts_to_sequences(corpus)
    
    tensor = [x for x in tensor if len(x) <= 15]
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

src_input = tensor[:, :-1]

tgt_input = tensor[:, 1:]

from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state = 42)

BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder = True)

class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences = True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 64
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer = optimizer)
model.fit(dataset, epochs = 10)

데이터 크기:  187088
Examples :
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]
[[   2   50    5 ...    0    0    0]
 [   2   17 2643 ...    0    0    0]
 [   2   35    7 ...   43    3    0]
 ...
 [   2    5  107 ...    0    0    0]
 [   2  261  200 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fc9d82b5ca0>
Epoch 1/10
610/610 [==============================] - 113s 178ms/step - loss: 3.5137
Epoch 2/10
610/610 [==============================] - 110s 180ms/step - loss: 3.0919
Epoch 3/10
610/610 [==============================] - 109s 179ms/step - loss: 2.9499
Epoch 4/10
610/610 [==============================] - 109s 179ms/step - loss: 2.8314
Epoch 5/10
610/610 [==============================] - 110s 180ms/step - loss: 2.7321
Epoch 6/10
610/610 [==============================] - 109s 179ms/step - loss: 2.6459
Epoch 7/10
610/610 [==============

## ...마치며

첫 번째로, preprocess_sentence를 이용하여 15 단어 이하인 문장만 사용하라고 되어 있어 문장을 split하여 리스트로 변환 후 숫자를 세어서 16개 이상인 것은 제외하도록 했는데, 두 번째로, 문장을 토크나이즈 하고 텐서에 저장할 때 15개 이하만 저장하도록 하는 방법으로 했을 때와 사용하는 데이터 수가 다르게 되어 첫번째로 사용한 방법이 맞는지 의문이 들지만, 그래도 목표였던 loss값이 2.2 이하로 떨어지게 하는 것에 성공했다.

문장을 만드는 것을 보면 같은 초기값을 줘도 출력 값이 다른 것을 확인할 수 있는데, 사용했던 데이터의 수가 달랐기 때문인 것 같다.

두 번째 방법을 보면 EPOCHS = 10을 거치며 오답률이 3.4376에서 시작해 2.1440으로 감소하고, 회당 111s가 걸린 것을 알 수 있다.
목표는 2.2 아래로 내리는 것이었는데, 충분히 달성한 것 같다.

#### embedding_size 와 hidden_size 값에 따른 loss값과 회당 걸리는 시간

위의 두 번째 방법에서 임베딩 사이즈와 히든 사이즈를 바꿔가며 실행 해보았다.

##### 임베딩 사이즈를 줄인 경우
* embedding_size = 128, hidden_size = 1024일 때, Epoch 1/10 - loss: 3.4776, Epoch 10/10 - loss: 2.2420으로 회당 111s의 시간이 걸렸고,

* embedding_size = 64, hidden_size = 1024일 때, Epoch 1/10 - loss: 3.5137, Epoch 10/10 - loss: 2.3620으로 회당 110s의 시간이 걸렸다.

##### 히든사이즈를 줄인 경우

* embedding_size = 256, hidden_size = 512일 때는, Epoch 1/10 - loss: 3.7359, Epoch 10/10 - loss: 2.4145으로 회당50s의 시간이 걸렸고,

* embedding_size = 256, hidden_size = 256일 때, Epoch 1/10 - loss: 3.8797, Epoch 10/10 - loss: 2.6682으로으로 회당 27s의 시간이 걸렸으며,

* embedding_size = 256, hidden_size = 128일 때는, Epoch 1/10 - loss: 4.1071, Epoch 10/10 - loss: 2.8133으로 회당18s의 시간이 걸렸다.


-> 이러한 경향으로 봤을 때, 히든사이즈 값이 작을 수록 loss값이 커지며, 회당 걸리는 시간이 짧아진다. 임베딩 사이즈는 값이 작을 수록 loss값이 커지지만 회당 걸리는 시간은 별 변화가 없는 것을 알 수 있다.